# Feature Engineering


In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/RecSys2024/
base_path = '/content/drive/MyDrive/RecSys2024/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Exp_Recsys2024_LGBM_train.ipynb  output			    Recsys2024_LGBM_test.ipynb
Exp_Recsys2024_preprocess.ipynb  preprocess2.ipynb	    Recsys2024_LGBM_train.ipynb
feature_output			 preprocess_article.ipynb   Recsys2024_preprocess.ipynb
input				 Recsys2024_EDA.ipynb
models				 Recsys2024_ensemble.ipynb


In [2]:
import pandas as pd
import polars as pl
import numpy as np
import pickle
import gc

In [3]:
DEBUG_MODE = True
#DEBUG_MODE = False
#DATA_TYPE = 'train'
DATA_TYPE = 'valid'
#DATA_TYPE = 'test'

# next_read_timeとnext_scroll_percentageがほぼ0のものを除く
REMOVE_NOISE = True
remove_noise_name = ''
if REMOVE_NOISE:
    remove_noise_name = 'wo_noise'

# train, validにはあるがtestにはない特徴量
ignore_col_train = ['next_scroll_percentage', 'next_read_time', 'article_id']
out_path = base_path + 'feature_output'

# Preprocessing

In [4]:
if DEBUG_MODE:
  input_path = base_path + 'input/ebnerd_small/'
else:
  input_path = base_path + 'input/ebnerd_large/'

In [5]:
!ls {input_path}

articles.parquet  train  validation


In [6]:
if DATA_TYPE == 'test':
    input_path = base_path + 'input/ebnerd_testset/'

if DATA_TYPE == 'train':
    df_impression = pl.read_parquet(input_path + 'train/behaviors.parquet')
elif DATA_TYPE == 'valid':
    df_impression = pl.read_parquet(input_path + 'validation/behaviors.parquet')
else:
    df_impression = pl.read_parquet(input_path + 'test/behaviors.parquet')

# Feature Engineering

In [7]:
def show_df(df, n=5):
    print(df.shape)
    display(df.head(n))


def expand_behavior(df, data_type='train'):
    df = df.explode("article_ids_inview").rename({"article_ids_inview": "article_id_inview"})
    if data_type in ('train', 'valid'):
        df = df.with_columns(
            (pl.col("article_id_inview").is_in(pl.col("article_ids_clicked"))).alias("clicked")
        )
        df = df.drop("article_ids_clicked")
    return df


def datetime_to_unix(df, col_name):
    return df.with_columns(
        pl.col(col_name).map_elements(lambda lst: [int(dt.timestamp()) for dt in lst])
    )


def user_features(df):
  return df.select([
      pl.col("user_id"),
      pl.col("impression_time_fixed").list.len().alias("u_history_len"),
      pl.col("impression_time_fixed").list.last().alias("u_impression_time_last"),
      pl.col("impression_time_fixed").list.mean().alias("u_impression_time_mean"),
      pl.col("read_time_fixed").list.last().alias("u_read_time_last"),
      pl.col("read_time_fixed").list.mean().alias("u_read_time_mean"),
      pl.col("scroll_percentage_fixed").list.last().alias("u_scroll_percentage_last"),
      pl.col("scroll_percentage_fixed").list.mean().alias("u_scroll_percentage_mean"),
  ])


def exact_first(df, col_name):

    new_col_name = f"{col_name}_first"
    df = df.with_columns(
        pl.col(col_name).list.first().alias(new_col_name)
    )
    return df

def pop_count_list(df, out_col_name, bins):

    pop_list = []
    for bin in bins:
        tmp = df.with_columns(
            (df["impression_time"] / bin).cast(pl.Int64).alias(f"time_bin_{bin}")
        ).group_by([f"time_bin_{bin}", "article_id"]).agg(
            pl.count().alias(out_col_name)
        )
        pop_list.append(tmp)
    return pop_list


In [8]:
print('impression_time min:', df_impression.select(pl.col("impression_time")).min().item())
print('impression_time max:', df_impression.select(pl.col("impression_time")).max().item())

df_impression = df_impression.with_columns(
    pl.col("article_ids_inview").list.len().alias("view_num").cast(pl.UInt8),
    #impression["impression_time"].dt.timestamp("ms") / 1000).cast(pl.Int64),
    (pl.col("impression_time").dt.timestamp("ms") / 1000).cast(pl.Int64),
)
# ほぼみられてない場合はnoiseとして削る。test dataには含まれていないfeature
if REMOVE_NOISE:
    df_impression = df_impression.filter(
        (pl.col('next_read_time') > 0.01) &
        ((pl.col("next_scroll_percentage") > 0.01) | pl.col("next_scroll_percentage").is_not_null())
    )

if DATA_TYPE in ('train', 'valid'):
    df_impression = df_impression.drop(ignore_col_train)
show_df(df_impression, 3)

impression_time min: 2023-05-18 07:00:01
impression_time max: 2023-05-25 06:59:58
(204205, 15)


impression_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,view_num
u32,i64,f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,u8
149474,1684914473,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,6
150528,1684913605,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,9
153068,1684912144,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,7


In [9]:
if DATA_TYPE == 'test':
  # testの場合はimpression_id=0が複数含まれており、user_idも入れないとuniqueにならないので入れる
  exp_col = ["article_ids_inview"]
  df_impression_article = df_impression.select(["impression_id", "user_id"] + exp_col)
else:
  exp_col = ["article_ids_inview", "article_ids_clicked"]
  df_impression_article = df_impression.select(["impression_id"] + exp_col)

df_impression = df_impression.drop(exp_col)

In [10]:
# expand list
df_impression_article = expand_behavior(df_impression_article, DATA_TYPE)
show_df(df_impression_article, 2)

(2244212, 3)


impression_id,article_id_inview,clicked
u32,i32,bool
149474,9778623,false
149474,9778682,false


In [11]:
# save
if DEBUG_MODE:
    df_impression.write_parquet(f"{out_path}/{remove_noise_name}small_{DATA_TYPE}_impression.parquet")
    df_impression_article.write_parquet(f"{out_path}/{remove_noise_name}small_{DATA_TYPE}_impression_article.parquet")
else:
    df_impression.write_parquet(f"{out_path}/{remove_noise_name}{DATA_TYPE}_impression.parquet")
    df_impression_article.write_parquet(f"{out_path}/{remove_noise_name}{DATA_TYPE}_impression_article.parquet")
del df_impression, df_impression_article
gc.collect()

0

## History features (key: user_id)

In [12]:
if DATA_TYPE == 'train':
    df_user = pl.read_parquet(input_path + 'train/history.parquet')
elif DATA_TYPE == 'valid':
    df_user = pl.read_parquet(input_path +  'validation/history.parquet')
else:
    df_user = pl.read_parquet(input_path + 'test/history.parquet')

In [13]:
df_user = datetime_to_unix(df_user, "impression_time_fixed")
show_df(df_user, 3)

(15143, 5)


user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[i64],list[f32],list[i32],list[f32]
13538,"[1682590663, 1682590681, … 1684355794]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[1682588418, 1682588433, … 1684343321]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[1682598644, 1682598694, … 1684321184]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"


In [14]:
# User * Article (過去に同一記事のclickがあった場合)
df_user_article = df_user.explode(["impression_time_fixed", "scroll_percentage_fixed", "article_id_fixed", "read_time_fixed"]).rename({"article_id_fixed": "article_id"})
df_user_article = df_user_article.group_by(["user_id", "article_id"]).agg([
    pl.col("impression_time_fixed").mean().alias("ua_impression_time_mean"),
    pl.col("impression_time_fixed").last().alias("ua_impression_time_last"),
    pl.col("scroll_percentage_fixed").mean().alias("ua_scroll_percentage_mean"),
    pl.col("scroll_percentage_fixed").last().alias("ua_scroll_percentage_last"),
    pl.col("read_time_fixed").mean().alias("ua_read_time_mean"),
    pl.col("read_time_fixed").last().alias("ua_read_time_last"),
    pl.count().alias("ua_count")
])
show_df(df_user_article, 3)

(2052877, 9)


user_id,article_id,ua_impression_time_mean,ua_impression_time_last,ua_scroll_percentage_mean,ua_scroll_percentage_last,ua_read_time_mean,ua_read_time_last,ua_count
u32,i32,f64,i64,f32,f32,f32,f32,u32
13538,9738569,1.6826e9,1682590681,35.0,35.0,12.0,12.0,1
13538,9738646,1.6826e9,1682607144,63.0,63.0,15.0,15.0,1
13538,9739274,1.6826e9,1682613518,69.0,69.0,22.0,22.0,1


In [15]:
df_user = user_features(df_user)
show_df(df_user, 2)

(15143, 8)


user_id,u_history_len,u_impression_time_last,u_impression_time_mean,u_read_time_last,u_read_time_mean,u_scroll_percentage_last,u_scroll_percentage_mean
u32,u32,i64,f64,f32,f32,f32,f32
13538,582,1684355794,1.6835e9,16.0,12.357388,100.0,40.382408
14241,179,1684343321,1.6834e9,12.0,43.318436,100.0,75.239517


In [16]:
# save
if DEBUG_MODE:
    df_user.write_parquet(f"{out_path}/small_{DATA_TYPE}_user.parquet")
    df_user_article.write_parquet(f"{out_path}/small_{DATA_TYPE}_user_article.parquet")
else:
    df_user.write_parquet(f"{out_path}/{DATA_TYPE}_user.parquet")
    df_user_article.write_parquet(f"{out_path}/{DATA_TYPE}_user_article.parquet")
del df_user, df_user_article
gc.collect()

0

## Article Features (key: article_id)

In [17]:
# read file
df_article = pl.read_parquet(input_path + '/articles.parquet')
# 使い道がなさそうなものは一旦けずる
ignore_cols = ["url"]
df_article = df_article.drop(ignore_cols)

# setで残しておくfeature, 後でuser素性と突き合わせて使う
#set_cols = ["subcategory", "topics", "entity_groups", "ner_clusters"]
#for i in set_cols:
#    df_article = df_article.with_columns(pl.col(i).map_elements(lambda x: set(x)).alias(f"{i}_set"))

# 後段で処理しやすいようにunixtimeに直しておく
df_article = df_article.with_columns(
    (df_article["published_time"].dt.timestamp("ms") / 1000).cast(pl.Int64),
    (df_article["last_modified_time"].dt.timestamp("ms") / 1000).cast(pl.Int64),
)

# 文字数だけをfeatureとして加える
str_cols = ["title", "subtitle", "body"]
for i in str_cols:
  df_article = df_article.with_columns(pl.col(i).str.len_bytes().cast(pl.UInt16).alias(f'{i}_len'))
  df_article = df_article.drop(i)

# listの最初の要素
first_element_list = ["ner_clusters", "entity_groups", "topics", "subcategory"]
for i in first_element_list:
  df_article = exact_first(df_article, i)

# 大事そうなやつはlastも加える
last_element_list = ["ner_clusters"]
for i in first_element_list:
  df_article = exact_first(df_article, i)

# listの要素数をfeatureとして加える
list_cols = ["image_ids", "ner_clusters", "entity_groups", "topics", "subcategory"]
for i in list_cols:
    df_article = df_article.with_columns(pl.col(i).list.len().cast(pl.UInt8).alias(f'{i}_len'))
    df_article = df_article.drop(i)

# fill null
num_features = ["total_inviews", "total_pageviews", "total_read_time", "sentiment_score"]
df_article = df_article.with_columns(pl.col(num_features).fill_null(0))

original_names = df_article.columns
new_names = [f"a_{name}" if name != 'article_id' else name for name in original_names]
df_article = df_article.rename(dict(zip(original_names, new_names)))
show_df(df_article, 3)

(20738, 24)


article_id,a_last_modified_time,a_premium,a_published_time,a_article_type,a_category,a_category_str,a_total_inviews,a_total_pageviews,a_total_read_time,a_sentiment_score,a_sentiment_label,a_title_len,a_subtitle_len,a_body_len,a_ner_clusters_first,a_entity_groups_first,a_topics_first,a_subcategory_first,a_image_ids_len,a_ner_clusters_len,a_entity_groups_len,a_topics_len,a_subcategory_len
i32,i64,bool,i64,str,i16,str,i32,i32,f32,f32,str,u16,u16,u16,str,str,str,i16,u8,u8,u8,u8,u8
3001353,1688019633,false,1157011605,"""article_defaul…",140,"""krimi""",0,0,0.0,0.9955,"""Negative""",29,151,1323,null,null,"""Kriminalitet""",null,1,0,0,2,0
3003065,1688019635,false,1148230620,"""article_defaul…",414,"""underholdning""",0,0,0.0,0.846,"""Positive""",25,119,769,null,null,"""Underholdning""",433,1,0,0,3,2
3012771,1688019639,false,1146493720,"""article_defaul…",142,"""sport""",0,0,0.0,0.8241,"""Negative""",33,79,1077,null,null,"""Erhverv""",196,1,0,0,5,2


In [18]:
# save
if DEBUG_MODE:
    df_article.write_parquet(f"{out_path}/small_{DATA_TYPE}_article.parquet")
else:
    df_article.write_parquet(f"{out_path}/{DATA_TYPE}_article.parquet")